# ADK 應用程式測試

本筆記本示範如何測試 ADK (代理開發套件) 應用程式。
它涵蓋了使用 Agent Engine 和 Cloud Run 進行本地和遠端測試。

> **注意**：本筆記本假設代理檔案儲存在 `app` 資料夾中。如果您的代理檔案位於不同的目錄，請相應地更新所有相關的檔案路徑和參考。

<img src="https://github.com/GoogleCloudPlatform/agent-starter-pack/blob/main/docs/images/adk_logo.png?raw=true" width="400">



## 安裝依賴項

In [ ]:
!pip install google-cloud-aiplatform --upgrade

### 匯入函式庫

In [1]:
import json

import requests
import vertexai.agent_engines

## 如果您正在使用 Agent Engine
更多文件請參閱 [Agent Engine 總覽](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### 遠端測試

In [4]:
# 替換為您的 Agent Engine ID
AGENT_ENGINE_ID = "projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"

In [5]:
remote_agent_engine = vertexai.agent_engines.get(AGENT_ENGINE_ID)

In [ ]:
for event in remote_agent_engine.stream_query(message="hi!", user_id="test"):
    print(event)

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

### 本地測試

您可以直接在您的環境中匯入 AgentEngineApp 類別。
要在本地執行代理，請按照以下步驟操作：
1. 確保您的環境中已安裝所有必要的套件
2. 建議的方法是使用 'uv' 工具建立的同一個虛擬環境
3. 您可以從代理的根目錄執行 'make install' 來設定此環境
4. 然後在您的 Jupyter 筆記本中選擇此核心（專案中的 .venv 資料夾），以確保所有依賴項都可用

In [2]:
# 如果您未使用 uv 建立的虛擬環境，請取消註解以下幾行
# import sys
# sys.path.append("../")

In [ ]:
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [ ]:
for event in agent_engine.stream_query(message="hi!", user_id="test"):
    print(event)

## 如果您正在使用 Cloud Run

#### 遠端測試

有關向 Cloud Run 服務驗證 HTTPS 請求的更多資訊，請參閱：
[Cloud Run 驗證文件](https://cloud.google.com/run/docs/triggering/https-request)

遠端測試涉及使用已部署的服務 URL 而不是 localhost。

驗證是使用 GCP 身份權杖而不是本地憑證來處理的。

In [20]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # 替換為您的 Cloud Run 服務 URL

您需要先建立一個會話

In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"{SERVICE_URL}/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"會話建立狀態碼: {session_response.status_code}")
print(f"會話建立回應: {session_response.json()}")
session_id = session_response.json()["id"]

然後您就可以傳送訊息了

In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = f"{SERVICE_URL}/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"訊息傳送狀態碼: {message_response.status_code}")

# 列印串流回應
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"收到的事件: {event}")

### 本地測試

> 您可以透過 `make local-backend` 指令在本地執行應用程式。

#### 建立一個會話
 建立一個帶有使用者偏好和狀態資訊的新會話


In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"http://127.0.0.1:8000/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"會話建立狀態碼: {session_response.status_code}")
print(f"會話建立回應: {session_response.json()}")
session_id = session_response.json()["id"]

#### 傳送一則訊息
向後端服務傳送一則訊息並接收串流回應


In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = "http://127.0.0.1:8000/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"訊息傳送狀態碼: {message_response.status_code}")

# 列印串流回應
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"收到的事件: {event}")